**1.** [**Project Plan**](#Project_Plan)<br>
**2.** [**Data Acquistion**](#acquire_data)<br>
**3.** [**Exploration**](#explore_data)<br>
**4.** [**Clustering**](#cluster_data)<br>
**5.** [**Modeling**](#model_data)<br>
**6.** [**Conclusions**](#conclusions)<br>

# Project Plan

### Background

> Using 2017 properties and prediction data from our Zillow database for single unit/single family homes, we were tasking with improving the log error (Zestimate)
> To accomplish this the team used clustering methodologies to find patterns in terms of which features had the greatest effect on log error
> Using the features which produced the best clusters, create a model to predict logerror

### Deliverables
**The deliverables for this project are the following data assets:**
- Report detailing our analysis in an .ipynb format
- Detailed README on a Github and repo containing all files for this project
- All .py files that are necessary to reproducible work

<a id='Project_Plan'></a>

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# general
import pandas as pd
import numpy as np

# explore/ stat
import scipy.stats as stats

# visuals
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

# support modules
import acquire
import wrangle
import prepare
import split_scale
import cluster
import model

# look at function for summarize plot
# import summarize
# import prepare
# import explore
# import split_scale

# modeling
from sklearn.cluster import KMeans, dbscan
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

# view full DF
pd.set_option('display.max_columns', None)

# Aquisition and Prep

**Retrieve from the Codeup Zillow_db:**

- Latest transactions occured within the year of 2017 for each property
- Logerror
- All fields related to each properties
    - Fields known to have greater than 60% nulls were excluded from the query
- Gather descriptions by joining description tables
- Only properties where latitude and longitude are not null
- Only properties where bedroom and bathroom count were not 0
- Only single family homes (SFR)

To create this dataframe use the get_zillow_data function from the acquire.py file

In [2]:
df = acquire.get_zillow_data()

df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,1998.0,485713.0,1023282.0,537569.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1967.0,88000.0,464000.0,376000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,1962.0,85289.0,564778.0,479489.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,1970.0,108918.0,145143.0,36225.0,1777.51,-0.103410,2017-01-01
4,10887214,3.0,3.0,1312.0,6037.0,34185120.0,-118414640.0,278581.0,1964.0,73681.0,119407.0,45726.0,1533.89,0.006940,2017-01-01


### The following transofmations we achieved through a series of functions:
> - ensure there are no nulls
> - remove all outliers are removed
> - create dervied features 
> - assigned intuitive names to each feature

- All datatypes are appropriate for our use

In [3]:
df.shape

(76725, 15)

In [4]:
df.isnull().sum()

parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedfinishedsquarefeet      11
fips                               0
latitude                           0
longitude                          0
lotsizesquarefeet               8152
yearbuilt                         65
structuretaxvaluedollarcnt        87
taxvaluedollarcnt                  1
landtaxvaluedollarcnt              1
taxamount                          5
logerror                           0
transactiondate                    0
dtype: int64

In [5]:
df = wrangle.handle_nulls(df)
df.isnull().sum()

parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
transactiondate                 0
dtype: int64

**Create synthetic features**

In [6]:
df = wrangle.prepare_zillow(df)

df = prepare.tax_rate(df)

In [7]:
# remove outliers in these coloumns
col_out = ["bathroomcnt", "bedroomcnt", "tax_rate", "calculatedfinishedsquarefeet", 
           "lotsizesquarefeet", "structuretaxvaluedollarcnt", "taxvaluedollarcnt", "landtaxvaluedollarcnt"]

df = prepare.remove_outliers_iqr(df, col_out)

- We noticed there were still a few features with some extreme values so we employed a more conventional method

In [8]:
# Additional outlier removal
df = df[((df.bathroomcnt <= 7) & (df.bedroomcnt <= 7) & 
         (df.bathroomcnt > 0) & 
         (df.bedroomcnt > 0) & 
         (df.calculatedfinishedsquarefeet < 7000) & 
         (df.tax_rate < .05)
        )]

In [9]:
df = prepare.bed_bath_ratio(df)
df = prepare.better_names(df)

In [10]:
df.head()

,longitude,latitude,age,bedroomcnt,bathroomcnt,square_footage,lot_size,house_value,land_value,full_value,tax_rate,Los_Angeles,Orange,Ventura,logerror,bed_bath_ratio
0,-117869207.0,33634931.0,19.0,4.0,3.5,3100.0,4506.0,485713.0,537569.0,1023282.0,0.010763,0,1,0,0.025595,1.142857
1,-119281531.0,34449266.0,50.0,2.0,1.0,1465.0,12647.0,88000.0,376000.0,464000.0,0.012225,0,0,1,0.055619,2.000000
2,-117823170.0,33886168.0,55.0,3.0,2.0,1243.0,8432.0,85289.0,479489.0,564778.0,0.011488,0,1,0,0.005383,1.500000
3,-118240722.0,34245180.0,47.0,4.0,3.0,2376.0,13038.0,108918.0,36225.0,145143.0,0.012247,1,0,0,-0.103410,1.333333
5,-118993991.0,34230044.0,35.0,3.0,2.0,1492.0,903.0,198640.0,132424.0,331064.0,0.010596,0,0,1,-0.020526,1.500000


In [11]:
df.shape

(59930, 16)

<a id='acquire_data'></a>

In [12]:
# parcelDensity30000 = cluster.get_pde(df,30000)

# plt.figure(figsize=(14,14))
# plt.axis("off")
# plt.title("Parcel Density Estimation")
# plt.scatter(df['longitude'].values, df['latitude'].values, c=parcelDensity30000,cmap='inferno', s=1, edgecolor='')
# plt.show()

# Exploration

During this section, the team identified patterns in the data which may help to isolate which features were most helpful in predicting churn.

- Hypothesis testing in this section assesses the level of correlation between our independent variables and between our independent and target variables
- Each descovery about a trend in the data is accompanied by a matching vizualization

Before diving into exploration

### Feature/Target Hypothesis
- $H_0$: Bedroom and bathroom count have no effect on logerror
- $H_a$: Bedroom and bathroom count do have an effect on logerror
- _alpha ($\alpha$): 1 - confidence level (95% confidence level -> $\alpha = .05$)_

### Feature/Feature Hypothesis
- $H_0$: Bedroom and bathroom count are not related
- $H_a$: Bedroom and bathroom count are related
- _alpha ($\alpha$): 1 - confidence level (95% confidence level -> $\alpha = .05$)_

### Value/Target Hypothesis
- $H_0$: Overall home value has no effect on logerror
- $H_a$: Over home value does have an effect on logerror
- _alpha ($\alpha$): 1 - confidence level (95% confidence level -> $\alpha = .05$)_

## Split and Scale

- The data was split into train and test dataframes (70% and 30% respectively). A random seed was set for reproducibility.
- Both sets of data were scaled using the MinMax scaler from SKlearn

In [13]:
train, test = split_scale.train_test(df)
print('train:', train.shape)
print('test:', test.shape)

train: (41951, 16)
test: (17979, 16)


In [14]:
train.head()

,longitude,latitude,age,bedroomcnt,bathroomcnt,square_footage,lot_size,house_value,land_value,full_value,tax_rate,Los_Angeles,Orange,Ventura,logerror,bed_bath_ratio
44758,-118476118.0,34061074.0,78.0,3.0,2.0,2898.0,7070.0,375122.0,244299.0,619421.0,0.013315,1,0,0,1.775059,1.5
76378,-117984001.0,34019979.0,61.0,4.0,2.0,1937.0,6048.0,179233.0,218384.0,397617.0,0.011867,1,0,0,-0.116786,2.0
3539,-118461331.0,33974843.0,48.0,4.0,5.0,3199.0,2884.0,309266.0,924804.0,1234070.0,0.012103,1,0,0,0.008819,0.8
35048,-117922000.0,33854800.0,54.0,2.0,2.0,1157.0,7201.0,81120.0,193295.0,274415.0,0.012080,0,1,0,-0.017483,1.0
31603,-117687326.0,33522157.0,31.0,3.0,3.0,1555.0,8600.0,167386.0,327332.0,494718.0,0.010014,0,1,0,0.006042,1.0


In [15]:
scaler, train_scaled, test_scaled = split_scale.min_max_scaler(train, test)

### Target Exploration: Logerror

In [16]:
# code to regulate the size of plots for the rest of notebook
plt.rc('figure', figsize=(13, 10))
plt.rc('font', size=13)

In [17]:
# # overall logerror distribution 
# sns.distplot(df.logerror)

In [18]:
# # absolute value of logerror
# sns.kdeplot(df.logerror.abs(), shade=True)
# plt.show()

### Let's look at the correlation between the target variable and all other variables

In [19]:
# # Heatmap to display correlation of the features and logerror.
# plt.figure(figsize=(14, 10))
# sns.heatmap(df.corr(), cmap='Greens', annot=True)

### The heatmap doesn't show any particulary high linear correlation but there could still be certain features that have more of an impact on the target variable

#### Let's take a closer look at some specific features :

#### Is there a relationship between Bathroom count or Bedroom count and Logerror?

In [20]:
# plt.figure(figsize=(14,8))
# with sns.color_palette('Blues'):
#     sns.barplot(x='bathroomcnt', y='logerror', data=train)
# plt.xlabel('Bathroom Count')
# plt.ylabel('Log Error')
# plt.title('Does bathroom count impact log error?')
# plt.show()

### It looks like homes with smaller bathroom counts tend to produce smaller log error,
> ### There are some major errors with home of 6 bathrooms
> ### This could possibly be a driver of log error. We'll keep this feature in mind when whe head to clustering and futher modeling

In [21]:
# plt.figure(figsize=(14,8))
# with sns.color_palette("Blues"):
#     sns.barplot(x='bedroomcnt', y='logerror', data=train)
# plt.xlabel('Bedroom Count')
# plt.ylabel('Log Error')
# plt.title('Does bedroom count impact log error?')
# plt.show()


### Bedroom count also shows some increse in logerror at higher counts
> #### These are the two home features that we can vizually see some variation in log error


In [22]:
# plt.figure(figsize=(14,8))
# sns.scatterplot(x='tax_rate', y='logerror', data=train, alpha=.4)
# plt.xlabel('Tax Rate')
# plt.ylabel('Log Error')
# plt.title('Does tax rate impact log error?')
# plt.show()

In [23]:
# plt.figure(figsize=(14,8))
# sns.scatterplot(x='square_footage', y='logerror', data=train, alpha=.4)
# plt.xlabel('Finished Square Feet')
# plt.ylabel('Log Error')
# plt.title('Does the finished square feet of a home impact log error?')
# plt.show()

<a id='explore_data'></a>

# Clustering

### Features were grouped into 4 categories:
> -  Physical position
> - Age of home
> -  Value of home
> - Home features

__We constructed several clusters using unique combinations of these features and evaluated them using ttests to determine significance of relationship between each group within the cluster and the target variable *logerror*.__

Our two best performing clusters are displayed below:

### First Cluster groups according to physical position and home features
> - #### Latitude
> - #### Longitude
> - #### Lot_size
> - #### Square_footage

In [24]:
cluster_vars = train_scaled[['latitude', 'longitude', 'lot_size', 'square_footage']]
cluster_col_name = 'location_size'
centroid_col_names = ['centroid_' + i for i in cluster_vars]

In [25]:
#determine what the best k (number of groups) is
# optimal_k = cluster.elbow_method(cluster_vars)

### Went with a k of 5

In [26]:
# Function to obtain:
# The train clusters with their observations,
# test clusters and their observations
# and a df of the number of observations per cluster on train
kmeans, train_clusters, test_clusters, cluster_counts = cluster.get_clusters_and_counts(5, ['latitude', 'longitude',
                                                                                           'lot_size', 'square_footage'], 
                                                                                        'location_size', train_scaled, test_scaled)

In [27]:
X_train_scaled = train_scaled[["latitude","longitude","square_footage", "lot_size", "full_value","age", "lot_size"]]
X_test_scaled = test_scaled[["latitude","longitude", "square_footage", "lot_size", "full_value","age", "lot_size"]]

In [28]:
# Function to obtain:
# dataframe of the train clusters with their observations, 
# test clusters and their observations
# and a df of the number of observations per cluster on train. 
X_train_scaled, train_scaled, X_test_scaled, test_scaled, centroids = cluster.append_clusters_and_centroids(
                                X_train_scaled, train_scaled, train_clusters, 
                                X_test_scaled, test_scaled, test_clusters, 
                                cluster_col_name, centroid_col_names, kmeans)

In [29]:
X_train_scaled.head()

,latitude,longitude,square_footage,lot_size,full_value,age,lot_size,location_size,centroid_latitude,centroid_longitude,centroid_lot_size,centroid_square_footage
44758,0.522594,0.520456,0.565886,0.288830,0.381740,0.401042,0.288830,3,0.364735,0.782326,0.316236,0.536854
76378,0.492830,0.776761,0.351616,0.245636,0.241751,0.312500,0.245636,0,0.466279,0.683158,0.250476,0.222903
3539,0.460140,0.528157,0.632999,0.111914,0.769668,0.244792,0.111914,3,0.364735,0.782326,0.316236,0.536854
35048,0.373196,0.809053,0.177703,0.294366,0.163993,0.276042,0.294366,4,0.247263,0.871520,0.249342,0.241904
31603,0.132273,0.931276,0.266444,0.353493,0.303035,0.156250,0.353493,4,0.247263,0.871520,0.249342,0.241904


### Second Cluster groups according to one home feature, the age of the home and one value feature
> - #### Lot_size
> - #### Age
> - #### full_value

In [30]:
# Define variables
cluster_vars = train_scaled[['lot_size', 'age', 'full_value']]
cluster_col_name = 'size_age_value'
centroid_col_names = ['centroid_' + i for i in cluster_vars]

In [31]:
#determine what the best k (number of groups) is
# optimal_k = cluster.elbow_method(cluster_vars)

### Again we went with a k of 5

In [32]:
# Function to obtain:
# The train clusters with their observations,
# test clusters and their observations
# and a df of the number of observations per cluster on train
kmeans, train_clusters, test_clusters, cluster_counts = cluster.get_clusters_and_counts(5, ['age',
                                                                                           'lot_size', 'full_value'], 
                                                                                        'size_age_value', train_scaled, test_scaled)

In [33]:
# Function to obtain:
# dataframe of the train clusters with their observations, 
# test clusters and their observations
# and a df of the number of observations per cluster on train. 
X_train_scaled, train_scaled, X_test_scaled, test_scaled, centroids = cluster.append_clusters_and_centroids(
                                X_train_scaled, train_scaled, train_clusters, 
                                X_test_scaled, test_scaled, test_clusters, 
                                cluster_col_name, centroid_col_names, kmeans)

In [34]:
X_train_scaled.head()

,latitude,longitude,square_footage,lot_size,full_value,age,lot_size,location_size,centroid_latitude,centroid_longitude,centroid_lot_size_x,centroid_square_footage,size_age_value,centroid_lot_size_y,centroid_age,centroid_full_value
44758,0.522594,0.520456,0.565886,0.288830,0.381740,0.401042,0.288830,3,0.364735,0.782326,0.316236,0.536854,3,0.347421,0.283276,0.307623
76378,0.492830,0.776761,0.351616,0.245636,0.241751,0.312500,0.245636,0,0.466279,0.683158,0.250476,0.222903,3,0.347421,0.283276,0.307623
3539,0.460140,0.528157,0.632999,0.111914,0.769668,0.244792,0.111914,3,0.364735,0.782326,0.316236,0.536854,2,0.199938,0.309452,0.590778
35048,0.373196,0.809053,0.177703,0.294366,0.163993,0.276042,0.294366,4,0.247263,0.871520,0.249342,0.241904,0,0.337996,0.267801,0.100313
31603,0.132273,0.931276,0.266444,0.353493,0.303035,0.156250,0.353493,4,0.247263,0.871520,0.249342,0.241904,4,0.143014,0.246705,0.235960


### Cluster Evaluation

In [35]:
cluster.test_sig(X_train_scaled.location_size, train_scaled)

,ttest,pval,stat_sig
0,4.683325,0.000003,True
1,-2.874978,0.004046,True
2,-0.035291,0.971849,False
3,1.093000,0.274474,False
4,-1.940458,0.052353,False


In [36]:
cluster.test_sig(X_train_scaled.size_age_value, train_scaled)

,ttest,pval,stat_sig
0,-3.906421,0.000094,True
1,1.353809,0.175863,False
2,3.854673,0.000116,True
3,-3.700716,0.000216,True
4,1.478603,0.139340,False


<a id='cluster_data'></a>

# Modeling

In [37]:
X_train = train_scaled[['lot_size','age','full_value','longitude','latitude','square_footage']]
y_train = train_scaled[['logerror']]

X_test = test_scaled[['lot_size','age','full_value','longitude','latitude','square_footage']]
y_test = test_scaled[['logerror']]

In [38]:
predictions = y_train[['logerror']]
predictions = predictions.rename(columns={'logerror': 'actual'})
predictions.head()

,actual
44758,0.824503
76378,0.574090
3539,0.590715
35048,0.587234
31603,0.590348


In [39]:
# create baseline
X_train = train_scaled[['lot_size','age','full_value','longitude','latitude','square_footage']]
y_train = train_scaled[['logerror']]

y_train['mean_logerror'] = y_train.logerror.mean()
rmse_baseline = np.sqrt(mean_squared_error(y_train.logerror, y_train.mean_logerror))
r2_baseline = r2_score(y_train.logerror, y_train.mean_logerror)

predictions['rsme_baseline'] = ('{:.4f}'.format(rmse_baseline))
print('This is the baseline dataset model performance')
print(f'RSME = {rmse_baseline:.4f}')
print(f'R2 score =  {r2_baseline:.4f}')

This is the baseline dataset model performance
RSME = 0.0216
R2 score =  0.0000


In [41]:
# create decision tree regressor
X_train = train_scaled[['lot_size','age','full_value','longitude','latitude','square_footage']]
y_train = train_scaled[['logerror']]

regressor = DecisionTreeRegressor(max_depth = 8, random_state=121)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_train)

rmse_dt_train = np.sqrt(mean_squared_error(y_train, y_pred))
r2_dt_train = r2_score(y_train, y_train_predicted)

predictions['rsme_dt'] = ('{:.4f}'.format(rmse_dt_train))
print('This is the train dataset model performance')
print(f'RSME = {rmse_dt_train:.4f}')
print(f'R2 score =  {r2_dt_train:.4f}')

NameError: name 'y_train_predicted' is not defined

In [42]:
# create random forest regressor
X_train = train_scaled[['lot_size','age','full_value','longitude','latitude','square_footage']]
y_train = train_scaled[['logerror']]

regressor = RandomForestRegressor(max_depth = 2, random_state=121, n_estimators=100)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_train)

rmse_rf_train = np.sqrt(mean_squared_error(y_train, y_pred))
r2_rf_train = r2_score(y_train, y_pred)

predictions['rsme_rf'] = ('{:.4f}'.format(rmse_rf_train))
print('This is the train dataset model performance')
print(f'RSME = {rmse_rf_train:.4f}')
print(f'R2 score =  {r2_rf_train:.4f}')

This is the train dataset model performance
RSME = 0.0216
R2 score =  0.0056


<a id='model_data'></a>

# Conclusions

<a id='conclusions'></a>